<a href="https://colab.research.google.com/github/kay-squared/PyTorchLearn/blob/main/model_functions_lib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch
from torch import nn

import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt
import numpy as np

!pip install  torchmetrics
import torchmetrics

from tqdm.auto import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 7.9 MB/s eta 0:00:00


# Model and function library

## 1. Some functions


In [7]:
def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn,
               device: torch.device):
  
  model.train()
  train_loss = 0
  train_acc=0
  for batch, (X,y) in enumerate(dataloader):

    X,y = X.to(device),y.to(device)
    
    y_pred = model(X)
    loss = loss_fn(y_pred,y)            # loss per batch (32 samples)
    train_loss+=loss
    train_acc += accuracy_fn(y_pred.argmax(dim=1),y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch%400 ==0:
      #print(f'Have looked at {batch * len(X)}/{len(train_dataloader.dataset)} samples')
      print(f"Current sample: {batch*len(X)}/{len(train_dataloader.dataset)}")
  
  
  train_loss /= len(train_dataloader)    # average loss per batch
  train_acc /= len(train_dataloader)
  

  return model, train_loss, train_acc


def test_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn,
               device: torch.device
              ):
  test_loss = 0
  test_acc = 0
  model.eval()
  with torch.inference_mode():
    for batch, (X,y) in enumerate(test_dataloader):
      X,y = X.to(device),y.to(device)
      y_pred = model(X)
      test_loss += loss_fn(y_pred,y)
      test_acc += accuracy_fn(y_pred.argmax(dim=1),y)

    test_loss /= len(test_dataloader)
    test_acc /= len(test_dataloader)
  
  return model, test_loss, test_acc

  



In [8]:
def accuracy_fn(y_true,y_pred):
  """
  ACC =(TP + TN) / (TP+TN+FP+FN)
  args> 
  y_true: true labels
  y_pred> argmax of the cross entropy output
  """
  correct = torch.eq(y_true, y_pred).sum().item()
  acc = (correct/len(y_pred))
  return acc


def eval_model(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               accuracy_fn,
               device):
  """ Returns a dictionary coontaining the results of model prediction on data_loader"""
  loss,acc= 0,0

  with torch.inference_mode():
    for X,y in tqdm(data_loader):
      X,y = X.to(device),y.to(device)
      #make predictions
      y_pred = model(X)

      # accumulate the loss and acc values per batch
      loss+= loss_fn(y_pred,y)
      acc+= accuracy_fn(y_true = y,
                       y_pred=y_pred.argmax(dim=1))
    
    loss /= len(data_loader)
    acc /= len(data_loader)

  return {"model_name": model.__class__.__name__,
          "model_loss": loss.item(),
          "model_acc": acc}


In [9]:
def plot_decision_boundary(model: torch.nn.Module, X: torch.Tensor, y: torch.Tensor):
    """Plots decision boundaries of model predicting on X in comparison to y.

    Source - https://madewithml.com/courses/foundations/neural-networks/ (with modifications)
    """
    # Put everything to CPU (works better with NumPy + Matplotlib)
    model.to("cpu")
    X, y = X.to("cpu"), y.to("cpu")

    # Setup prediction boundaries and grid
    x_min, x_max = X[:, 0].min() - 0.1, X[:, 0].max() + 0.1
    y_min, y_max = X[:, 1].min() - 0.1, X[:, 1].max() + 0.1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 101), np.linspace(y_min, y_max, 101))

    # Make features
    X_to_pred_on = torch.from_numpy(np.column_stack((xx.ravel(), yy.ravel()))).float()

    # Make predictions

    model.eval()
    with torch.inference_mode():
        y_logits = model(X_to_pred_on)

    # Test for multi-class or binary and adjust logits to prediction labels
    if len(torch.unique(y)) > 2:
        y_pred = torch.softmax(y_logits, dim=1).argmax(dim=1)  # mutli-class
    else:
        y_pred = torch.round(torch.sigmoid(y_logits))  # binary

    # Reshape preds and plot
    y_pred = y_pred.reshape(xx.shape).detach().numpy()
    plt.contourf(xx, yy, y_pred, cmap=plt.cm.RdYlBu, alpha=0.7)
    plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.RdYlBu)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())

def plot_learning_curve(epoch_count, train_loss_list, test_loss_list):
  """ plots learning curve
      plot data to be accumulated in the training loop at defined intervals (args):
        epoch_count
        train_loss_list
        test_loss_list
  """

  plt.figure(figsize=(10,7))
  plt.plot(epoch_count,train_loss_list,c="b", label="Training loss")
  plt.plot(epoch_count,test_loss_list,c="orange", label="Test loss")
  plt.legend(prop={"size": 14})

  